In [ ]:
from dateutil.parser import parse
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

from src.data_handler import DataHandler
from src.analysis_logic import FitMethods

sns.set_theme()

# Method 1: Load all measurements and filter

Use this method when exploring data

## Set up input file paths
- Give it the measurement folder name and parameters to use
- The measurement folder must be located under the data folder specified in `parameters.py` (eg. `Z:/Data/20220621_FR0612-F2-2S6_uhv`)

In [ ]:
tip_2S6 = DataHandler(data_folder="C:\\Data", figure_folder="C:\\QudiHiraAnalysis", measurement_folder="20220621_FR0612-F2-2S6_uhv")

## Plot all confocal images for a given measurement
- Automatically walk through measurement folders and extract a list of confocal images.
- Each list element is an instance of `MeasurementDataclass` with attributes like `data`, `params` etc.
- For additional info stored in the filename, like height, power etc., call `get_param_from_filename(unit)` with the unit of the data. e.g. to get the power from "20220501_confocal_50mW", the function call will be `get_param_from_filename(unit="mW") => 50`.
- Output folder is automatically created from the output data path `Parameters` and the measurement folder name. For ease of use in presentations/publications etc. images are saved as `.png`, `.pdf`, `.svg` and `.jpg`.

In [ ]:
tip_2S6_confocal_list = tip_2S6.load_measurements_into_dataclass_list(measurement_str="Confocal")

# Set up matplotlib figure
fig, ax = plt.subplots(nrows=len(tip_2S6_confocal_list))

# Loop over all confocal images
for idx, confocal in enumerate(tip_2S6_confocal_list):
    # Plot each confocal image on a subplot row
    ax[idx].imshow(confocal.data)
    # Extract the power param (in mW units) from the name of file and set it to title
    power = confocal.get_param_from_filename(unit="mW")
    ax[idx].set_title(f"Laser power = {power}")

# Save output image
tip_2S6.save_figures(fig, filename="compare_confocals_at different_laser_powers")

### Filter out specific confocal images to plot
Since the confocal images are stored in the list, the required images can just be indexed out

In [ ]:
# Filter out the confocal images from index 10 to 20
filtered_list = tip_2S6_confocal_list[10:20]

Alternatively, any of the attributes can be used to filter images

In [ ]:
# Filter based on timestamp, filter all confocal images taken between 10th and 15th May 2022
timestamp_filtered_list = [confocal for confocal in tip_2S6_confocal_list if parse("10 May 2022") < confocal.timestamp < parse("15 May 2022")]

# Filter based on laser power, filter all confocal images with laser powers between 25 and 75 mW
param_filtered_list = [confocal for confocal in tip_2S6_confocal_list if 25 < confocal.get_param_from_filename(unit="mW") < 75]

# Filter based on image counts, filter all confocal images with maximum counts above 100,000 cps
data_filtered_list = [confocal for confocal in tip_2S6_confocal_list if np.max(confocal.data) > int(1e5)]

To reference a specific confocal image or a specific set of confocal images

In [ ]:
# via index
index_confocal = tip_2S6_confocal_list[10]

# via timestamp, images taken on 1 May 2022
single_timestamp_confocal = [confocal for confocal in tip_2S6_confocal_list if confocal.timestamp.date == parse("1 May 2022").date]

# via set of timestamps
timestamps = [
    parse("1 May 2022 10:30"),
    parse("1 May 2022 10:35"),
    parse("1 June 2022 12:15"),
]
multiple_timestamp_confocals = [confocal for confocal in tip_2S6_confocal_list if confocal.timestamp == any(timestamps)]

## Plot all Rabi oscillations for a given measurement and fit each one to an exponentially decaying sinusoid

Rabi oscillations fit to

$$ P_{m_s = 0}(\tau; T_{1\rho}, \omega_R) \propto e^{(-T_{1\rho} \tau)} \sin{(\omega_R \tau)} $$

In [ ]:
tip_2S6 = DataHandler(measurement_folder="20220621_FR0612-F2-2S6_uhv")

tip_2S6_rabi_list = tip_2S6.load_measurements_into_dataclass_list(measurement_str="Rabi")
filtered_rabi_list = [rabi for rabi in tip_2S6_rabi_list if
                      parse("10 July 2022 13:30") < rabi.timestamp < parse("10 July 2022 17:30")]

fig, ax = plt.subplots(nrows=len(filtered_rabi_list))

for idx, rabi in enumerate(filtered_rabi_list):
    x, y = rabi.pulsed.measurement.data["t(ns)"], rabi.pulsed.measurement.data["spin_state"]
    fit_x, fit_y = rabi.fit(x=x, y=y, fit_function=FitMethods.sinedoublewithexpdecay)

    ax[idx].plot(x, y, ".")
    ax[idx].plot(fit_x, fit_y, "-")
    ax[idx].set_title(f"Power = {rabi.get_param_from_filename(unit='dBm')}, "
                      f"T1rho = {rabi.fit_result.params['Lifetime']}")

tip_2S6.save_figures(fig, filename="compare_rabi_oscillations_at different_powers")

# Method 2: Load specific measurements

Use this method when you know what measurements you are interested in. Useful for preparing images for papers and presentations.

In [ ]:
tip_3P16 = DataHandler(measurement_folder="20220304_FR0612-F2-3P16")
# Give a list of specific measurements to be loaded
tip_3P16_measurements = tip_3P16.load_measurement_list_into_dataclass_list([
        r"PulsedMeasurement\20220316-1434-53_rabi_broken_cable_22dBm_pulsed_measurement",
        r"PulsedMeasurement\20220316-1545-42_ramsey_broken_cable_22dBm_pulsed_measurement_fig",
        r"Counter\20220316-2219-26_count_trace_fig",
        r"Counter\20220316-2246-44_count_trace_background_fig",
        r"PulsedMeasurement\20220315-2050-39_odmr_22dBm_600ns_pulsed_measurement_fig",
    ]
)

## Rabi

In [ ]:
rabi = tip_3P16_measurements[0]

fig, ax = plt.subplots(figsize=(5, 3))

# Recompute spin state and plot
x = rabi.data["Controlled variable(s)"] * 1e9
y, yerr = rabi.analysis.analyse_mean_norm(rabi.pulsed.laser_pulses.data, signal_start=150e-9, signal_end=250e-9, norm_start=1000e-9, norm_end=2000e-9)
ax.errorbar(x, y, yerr=yerr, fmt=".", capsize=3, markeredgewidth=1)

# Fit to sine with stretched exponentional decay
fit_x, fit_y, result = rabi.analysis.perform_fit(x=x, y=y, fit_function=FitMethods.sinestretchedexponentialdecay)
ax.plot(fit_x, fit_y)

ax.set_xlabel(r"$\tau_{RF}$ (ns)")
ax.set_ylabel("$P (m_s = 0)$")
ax.set_title(r"$T_{1\rho} \approx $" + f"{round(result.values['lifetime'] / 1e3)} μs")

tip_3P16.save_figures(fig, filename=rabi.filename)

### Recompute spin state

- Using signal window mean
- Using signal window mean normalized to normalization window
- Using signal window mean referenced to normalization window

In [ ]:
x = rabi.data["Controlled variable(s)"]

# Analyze using mean, signal from 150 to 400 ns
y_mean = rabi.analysis.analyse_mean(rabi.pulsed.laser_pulses.data, signal_start=150e-9, signal_end=400e-9)

# Analyze using mean normalized, with normalization from 2000 to 3000 ns
y_mean_norm = rabi.analysis.analyse_mean_norm(rabi.pulsed.laser_pulses.data, signal_start=150e-9, signal_end=400e-9, norm_start=2000e-9, norm_end=3000e-9)

# Analyze using mean reference, with reference from 2000 to 3000 ns
y_mean_ref = rabi.analysis.analyse_mean_reference(rabi.pulsed.laser_pulses.data, signal_start=150e-9, signal_end=400e-9, norm_start=2000e-9, norm_end=3000e-9)

## Ramsey

In [ ]:
ramsey = tip_3P16_measurements[1]

fig, ax = plt.subplots(figsize=(5, 3))

x = ramsey.data["Controlled variable(s)"] * 1e6
y1, y1err = ramsey.data["Signal"], ramsey.data["Error"]
y2, y2err = ramsey.data["Signal2"], ramsey.data["Error2"]
y_diff =  y1 - y2
y_differr = np.sqrt(np.square(y1err) + np.square(y2err))
ax.errorbar(x, y_diff, yerr=y_differr, fmt="o", capsize=4, markeredgewidth=1)

fit_x, fit_y, result = ramsey.analysis.perform_fit(x=x, y=y_diff, fit_function=FitMethods.sinedoublewithexpdecay)
ax.plot(fit_x, fit_y)

ax.set_xlabel(r"$\tau$ (μs)")
ax.set_ylabel(r"$\Delta \left( \hat{P} \left( \frac{\pi}{2} \right) - \hat{P} \left( \frac{3\pi}{2} \right) \right)$")
ax.set_title(r"$T_2^* \approx $" + f"{round(result.values['lifetime'])} μs")

tip_3P16.save_figures(fig, filename=ramsey.filename)

## Timetrace

In [1]:
from scipy.signal import savgol_filter

In [ ]:
fig, ax = plt.subplots(nrows=2, sharex="all", figsize=(5, 3))
titles = ["NV", "Background"]

for idx, counter in enumerate(tip_3P16_measurements[2:4]):
    if idx == 0:
        # Roughly match total time of measurement window
        x, y = counter.data["Time (s)"][:len(counter.data)//2], counter.data["Signal0 (counts/s)"][:len(counter.data)//2] / 1e3
    else:
        x, y = counter.data["Time (s)"], counter.data["Signal0 (counts/s)"] / 1e3
    ax[idx].plot(x, y)
    # Add Savitzky-Golay filtered timetrace
    ax[idx].plot(x, savgol_filter(y, 21, 1))
    ax[idx].set_title(titles[idx])

ax[-1].set_xlabel("Time (s)")
fig.supylabel("Fluorescence (kc/s)")

tip_3P16.save_figures(fig, filename="timetrace_nv_vs_background")

## ODMR

In [ ]:
odmr = tip_3P16_measurements[4]

fig, ax = plt.subplots(figsize=(5, 3))

x = odmr.data["Controlled variable(Hz)"] / 1e9
y, yerr = odmr.analysis.analyse_mean_norm(odmr.pulsed.laser_pulses.data, signal_start=150e-9, signal_end=250e-9, norm_start=1000e-9, norm_end=2000e-9)
ax.errorbar(x, y, yerr=yerr, fmt=".", capsize=3, markeredgewidth=1)

fit_x, fit_y, result = odmr.analysis.perform_fit(x=x, y=y, fit_function=FitMethods.lorentziandouble, estimator="dip")
ax.plot(fit_x, fit_y)

ax.set_xlabel(r"$\omega_{RF}$ (GHz)")
ax.set_ylabel("$P (m_s = 0)$")

tip_3P16.save_figures(fig, filename=odmr.filename)